In [1]:
# Wavefunction generation
from pyscf import mcscf, fci, lib 
from pyscf import gto, scf, tools, dft
from pyscf.dft import numint
import api as pyq 
import importlib
import os
import h5py
import pdb
import pyscf
import numpy as np
from pyscf.scf.hf import dump_scf_summary
print(pyq.__file__)
print(pyscf.__file__)

/Users/ksu/Documents/GitHub/pyqmc/pyqmc/api.py
/usr/local/anaconda3/lib/python3.9/site-packages/pyscf/__init__.py


In [2]:
mol = gto.M(atom="H 0. 0. 0.;", spin=1, basis=f'ccecpccpvqz', unit='bohr')
mf = dft.UKS(mol)
dm = mf.init_guess_by_atom()
mf.kernel(dm, xc='LDA,VWN')

converged SCF energy = -0.478375165323873  <S^2> = 0.75  2S+1 = 2


-0.478375165323873

In [3]:
dm = mf.make_rdm1()
kin = mf.mol.intor_symmetric('int1e_kin')
ein = mf.mol.intor_symmetric('int1e_nuc')
j, k= mf.get_jk(mf.mol, dm)

In [4]:
mol = mf.mol
coords = mf.grids.coords
weights = mf.grids.weights
ao_value = numint.eval_ao(mol, coords)

In [5]:
#Ekin
ekin = np.einsum('ij,sji->', kin, dm).real
#EI
ei = np.einsum('ij,sji->', ein, dm).real
#EJ
ej = 0.5*np.einsum('sij,sji->', j, dm).real

#EXC  1st way
a=10
delta = 0.2
xs = np.arange(-a, a, delta)
ys = np.arange(-a, a, delta)
zs = np.arange(-a, a, delta)
points = lib.cartesian_prod([xs, ys, zs])
ao_value2 = numint.eval_ao(mol, points)
rho_u = numint.eval_rho(mol, ao_value2, dm[0], xctype='LDA')
rho_d = numint.eval_rho(mol, ao_value2, dm[1], xctype='LDA')
excd, vxcs  = dft.libxc.eval_xc('LDA,VWN', np.array([rho_u.T, rho_d.T]), spin=1)[:2]
exc = np.einsum('i,i->', excd, (rho_u+rho_d))*delta**3

#EXC 2nd way
grids = mf.grids
nelec, exc2, vxc2 = dft.numint.nr_vxc(mol, grids, 'lda,vwn', dm, spin=1)

#ENUC
enuc = mf.energy_nuc()
print('e1', ekin + ei, 'coul', ej, 'exc', exc, 'exc2', exc2,'nuc', enuc)
#Compare to PYSCF
print(mf.scf_summary)

e1 -0.4987558744005207 coul 0.29863682280874154 exc -0.27825791410815714 exc2 -0.27825611373209375 nuc 0
{'e1': -0.49875587440052077, 'coul': 0.29863682280874154, 'exc': -0.27825611373209375, 'nuc': 0}


In [6]:
np.sum(vxcs[0], axis=1).shape

(1000000,)

In [7]:
j.shape

(2, 30, 30)

In [230]:
# # Validity check
# print(np.einsum('sij,sji,ik,ki->', kin, dm, ao_value2.T, ao_value2)*delta**3, ekin)
# print(np.einsum('ij,ji,ik,ki->', ein, dm, ao_value2.T, ao_value2)*delta**3, ei)
# print(0.5*np.einsum('ij,ji,ik,ki->', j, dm, ao_value2.T, ao_value2)*delta**3, ej)

# rho = numint.eval_rho(mol, ao_value2, dm, xctype='LDA')
# excd, vxcs  = dft.libxc.eval_xc('LDA,VWN', rho)[:2]
# print(np.einsum('i,i->', excd, rho)*delta**3, exc)

In [231]:
# excd, vxcs  = dft.libxc.eval_xc('LDA', rho)[:2]

In [232]:
# vxcs/rho**(1./3)

In [233]:
# points[0]

In [234]:
# np.einsum('pij,ij->p', mol.intor('int1e_grids', grids=np.array([points[1]])), mf.make_rdm1())

In [235]:
# with mol.with_rinv_origin(points[1]):
#     rinv = mol.intor('int1e_rinv')
#     print(np.einsum('ij,ij', rinv, dm))

In [236]:
# points[1]

In [237]:
# rho = numint.eval_rho(mol, ao_value2, dm, xctype='LDA')
# dft.libxc.eval_xc('LDA,VWN', rho)[:2]

In [294]:
vkin = np.einsum('ij,ji,ik,ki->k', kin, dm, ao_value2.T, ao_value2)
print(vkin.shape)
print(np.sum(vkin)*delta**3)

(1000000,)
1.1776771532722528


In [333]:
vei = np.einsum('ij,ji,ik,ki->k', ein, dm, ao_value2.T, ao_value2)
print(vei.shape)
print(np.sum(vei)*delta**3)

(1000000,)
-3.8138916711456194


In [296]:
vj = 0.5*np.einsum('ij,ji,ik,ki->k', j, dm, ao_value2.T, ao_value2) 
print(vj.shape)
print(np.sum(vj)*delta**3)

(1000000,)
1.3698681067334841


In [289]:
vexc = np.einsum('i,i->i', excd, rho)
print(vexc.shape)
print(np.sum(vexc)*delta**3)

(1000000,)
-0.684358302652905


In [290]:
np.sum(vkin + vei + vj + vexc)*delta**3 + enuc, mf.energy_tot()

(-1.1173713804594554, -1.117365504801064)

In [331]:
vkin + vei + vj + vexc

array([-3.60535619e-21, -5.95504048e-21, -9.73873612e-21, ...,
       -6.26612889e-19, -4.10677726e-19, -2.66509953e-19])

In [343]:
eeee = np.einsum('ij,jk,j,ki->i', ao_value2, dm, mf.mo_energy*mf.mo_occ, ao_value2.T)

In [344]:
eeee.shape

(1000000,)

In [345]:
eeee

array([-4.01586267e-21, -6.65107739e-21, -1.09063696e-20, ...,
       -1.88627536e-19, -1.19650635e-19, -7.51463683e-20])

In [24]:
from pyscf import gto, dft
import numpy as np
  
# Create a variable that stores the x,y,z atomic coordinates
atomic_coordinates = '''
H 0.00000  0.00000  0.00000 
H 0.74000  0.00000  0.00000
'''
  
mol = gto.Mole() # Create a Mole object
mol.atom = atomic_coordinates # Specify the coordinates
mol.basis = 'ccecpccpvqz' # Specify the basis set
mol.build() # Build the Mole object
  
mf = dft.KS(mol) # Create a KS-DFT object and pass the mol object as argument
mf.xc = 'pbe' # shorthand for pbe,pbe [PBE exchange and PBE correlation]
mf.kernel() # Run the DFT calculation
 
# MO Coefficients of the occupied orbitals
occ_orbs = mf.mo_coeff[:, mf.mo_occ > 0.]
# Generate some grids (we could have also used the already generated grids though)
grids = dft.gen_grid.Grids(mol)
grids.build()
# Get the weights corresponding to the grid points
weights = grids.weights
 
# Get the first derivatives of the atomic orbitals at grid points
ao1 = dft.numint.eval_ao(mol, grids.coords, deriv=1, non0tab=grids.non0tab)
# The following gives the non-negative kinetic energy density at grid points
ts = 0.5 * np.einsum('xgp,pi,xgq,qi->g', ao1[1:,:,:], occ_orbs, ao1[1:,:,:], occ_orbs)
 
Ts = np.einsum('g,g->', weights, ts)
print('Numerical KE', Ts)
 
# Calculate KE analytically from the analytical kinetic potential matrix and contracting with the density matrix
Ts_ao = mol.intor('int1e_kin')
Ts_analyt = np.einsum('ui,uv,vi->', occ_orbs, Ts_ao, occ_orbs)
print('Analytical KE', Ts_analyt)
 
# Way 3 
dm = mf.make_rdm1()
kin = mf.mol.intor_symmetric('int1e_kin')
print(0.5*np.einsum('ij,ji->', kin, dm).real)
print('Diff b/w Analytical and Numerical Ts', np.abs(Ts-Ts_analyt))

converged SCF energy = -1.16580013055239
Numerical KE 0.5694138224927637
Analytical KE 0.5694138198375434
0.5694138198375437
Diff b/w Analytical and Numerical Ts 2.655220376901468e-09


In [18]:
occ_orbs*occ_orbs

array([[1.67004206e-01],
       [2.90775565e-02],
       [2.98656180e-04],
       [1.35766228e-33],
       [1.27106635e-34],
       [1.67004206e-01],
       [2.90775565e-02],
       [2.98656180e-04],
       [3.93258441e-33],
       [1.08175076e-34]])

In [22]:
occ_orbs.shape

(10, 1)